# Handwritten Digit Recognition using Neural Network
This notebook implements a simple neural network from scratch to classify handwritten digits (0-9) using the MNIST dataset.
We will use **NumPy** for matrix operations and **Matplotlib** for visualization.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Load the dataset (MNIST Handwritten Digits)
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
data = np.array(data)
m, n = data.shape  # m = total samples, n = features + label

# Shuffle the data to randomize training and development sets
np.random.shuffle(data)  

# Split into development and training sets
data_dev = data[0:1000].T  # First 1000 samples for development (testing)
Y_dev = data_dev[0]  # Labels
X_dev = data_dev[1:n] / 255.  # Normalize pixel values (0-255 -> 0-1)

data_train = data[1000:m].T  # Remaining for training
Y_train = data_train[0]
X_train = data_train[1:n] / 255.

_, m_train = X_train.shape  # Number of training samples
print("Training Data Shape:", X_train.shape)

## Initialize Neural Network Parameters
The neural network consists of:
- **Input Layer (784 neurons)**: Each digit is a 28x28 image, flattened to 784 pixels.
- **Hidden Layer (10 neurons)**: Using ReLU activation.
- **Output Layer (10 neurons)**: Using Softmax activation (for 10 classes: digits 0-9).

In [ ]:
def init_params():
    """Initialize weights and biases with small random values."""
    W1 = np.random.rand(10, 784) - 0.5  # Hidden layer weights
    b1 = np.random.rand(10, 1) - 0.5  # Hidden layer bias
    W2 = np.random.rand(10, 10) - 0.5  # Output layer weights
    b2 = np.random.rand(10, 1) - 0.5  # Output layer bias
    return W1, b1, W2, b2

## Activation Functions
- **ReLU (Rectified Linear Unit)**: For hidden layer activation.
- **Softmax**: Converts output into probabilities.

In [ ]:
def ReLU(Z):
    return np.maximum(Z, 0)  # ReLU activation

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z), axis=0, keepdims=True)  # Softmax activation

## Forward Propagation
Computes activations for each layer using the initialized parameters.

In [ ]:
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

## Backward Propagation
Computes gradients and updates weights using gradient descent.

In [ ]:
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = np.eye(10)[Y].T  # Convert labels to one-hot encoding
    
    dZ2 = A2 - one_hot_Y  # Error in output layer
    dW2 = 1 / m_train * dZ2.dot(A1.T)
    db2 = 1 / m_train * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * (Z1 > 0)  # ReLU derivative
    dW1 = 1 / m_train * dZ1.dot(X.T)
    db1 = 1 / m_train * np.sum(dZ1, axis=1, keepdims=True)
    
    return dW1, db1, dW2, db2

## Training the Neural Network
Using **gradient descent** to update weights iteratively.

In [ ]:
def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1 -= alpha * dW1
        b1 -= alpha * db1
        W2 -= alpha * dW2
        b2 -= alpha * db2
        if i % 10 == 0:
            print("Iteration:", i, "Accuracy:", np.mean(np.argmax(A2, axis=0) == Y))
    return W1, b1, W2, b2

## Training the Model
We train the neural network using gradient descent for 500 iterations with a learning rate of 0.1.

In [ ]:
# Train the model
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, alpha=0.1, iterations=500)

## Testing the Model
We define functions to make predictions and evaluate accuracy.

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    return np.argmax(A2, axis=0)  # Get predicted labels

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size  # Compute accuracy

### Evaluate on Training Set

In [ ]:
train_predictions = make_predictions(X_train, W1, b1, W2, b2)
train_accuracy = get_accuracy(train_predictions, Y_train)
print("Training Accuracy:", train_accuracy)

### Evaluate on Development (Testing) Set

In [ ]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
dev_accuracy = get_accuracy(dev_predictions, Y_dev)
print("Development Set Accuracy:", dev_accuracy)

### Visualizing a Prediction
We can visualize a sample prediction from the development set.

In [ ]:
def test_prediction(index, W1, b1, W2, b2):
    current_image = X_dev[:, index, None]
    prediction = make_predictions(current_image, W1, b1, W2, b2)
    label = Y_dev[index]
    
    print("Prediction:", prediction)
    print("Label:", label)
    
    current_image = current_image.reshape((28, 28)) * 255  # Reshape and scale image
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
# Test with a random image
test_prediction(0, W1, b1, W2, b2)  # Change index to test different images